## Load librarys

In [1]:
import os

import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization
from collections import Counter
import keras
from keras import layers, regularizers

In [2]:
%load_ext watermark
%watermark --iversions

keras : 3.13.1
numpy : 2.3.5
pandas: 2.3.3



## Load data

In [2]:
x_train_Znorm = pd.read_csv('data/x_train_Znorm.csv')
x_val_Znorm = pd.read_csv('data/x_val_Znorm.csv')
y_train = np.load('data/y_train.npy')
y_val = np.load('data/y_val.npy')

## Train DNN

In [3]:
# checkpoints:
def create_model():
    model=keras.Sequential([
        layers.Input(shape=(72,)),    
        layers.Dense(256, activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-2), bias_regularizer=regularizers.L2(1e-2),
        activity_regularizer=regularizers.L2(1e-5)),
        BatchNormalization(),
        layers.Dropout(0.8),  
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4),
        activity_regularizer=regularizers.L2(1e-5)),
        BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(6, activation='softmax')
    ])
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])
    
    return model

def class_w(y_train):
    y_train=np.argmax(y_train, axis=1)
    class_counts = Counter(y_train)
    total_samples = len(y_train)
    class_w = {cls: total_samples / (len(class_counts) * count) for cls, count in class_counts.items()}
    return class_w

In [ ]:
i=0
j=0
acc=0.96
acc_list=[]
while i<100:
    filepath=f'data/best_model_v2_{i}.keras'
    reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.98, patience=20, min_lr=0.0000001,mode='auto')
    checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=0, save_best_only=True, mode='max')
    callbacks_list = [checkpoint,reduce_lr]
    cl_w = class_w(y_train)
    history=create_model().fit(x_train_Znorm,y_train,batch_size=50,epochs=300,validation_data=(x_val_Znorm,y_val),callbacks=callbacks_list, class_weight=cl_w)
    acc_list.append(max(history.history['val_categorical_accuracy']))
    if max(history.history['val_categorical_accuracy'])>acc:
        np.savez(f'data/history_v2_{i}.npz', 
         data=pd.DataFrame(history.history).to_numpy().astype(np.float32),
         columns=list(pd.DataFrame(history.history).columns))
        print(f'Model {i} done during {j} attempts')
        i+=1
    np.save('data/acc_list',acc_list)
    j+=1